In [191]:
import requests
import pandas as pd
from config import fund_dict
import xml.etree.ElementTree as ET
import xmltodict
import traceback

## Scrapping Data

In [192]:
_BASE_URL_SEC_API = "https://data.sec.gov"
_BASE_URL_XBRL = f"{_BASE_URL_SEC_API}/api/xbrl"

# SEC API endpoints as documented here:
# https://www.sec.gov/edgar/sec-api-documentation
BASE_URL_SUBMISSIONS = f"{_BASE_URL_SEC_API}/submissions"
cik = '0001535392'
accessionNumber='000117266122002482'
BASE_URL_FORM_TABLE = f"{_BASE_URL_SEC_API}/Archives/edgar/data/{cik}/{accessionNumber}/xslForm13F_X01/infotable.xml"
#https://www.sec.gov/edgar/browse/?CIK=1535392
#accession numner: 000117266122002482
print('https://www.sec.gov/Archives/edgar/data/1535392/000117266122002482/xslForm13F_X01/infotable.xml')

https://www.sec.gov/Archives/edgar/data/1535392/000117266122002482/xslForm13F_X01/infotable.xml


In [201]:
#https://www.sec.gov/os/webmaster-faq#privacy-enhanced-message
headers = {'User-Agent':
           'StockFries stockfries@gmail.com'}
api_endpoint = f"{BASE_URL_SUBMISSIONS}/CIK{cik}.json"
print(api_endpoint)
r = requests.get(url=api_endpoint, headers=headers)
print(r.json()['filings'].keys())
print(r.json()['filings']['recent'].keys())
#print(r.json()['filings']['recent']['primaryDocument'])
print(r.json()['filings']['recent']['filingDate'][0])
print(r.json()['filings']['recent']['reportDate'][0])
r.json()['filings']['recent']['accessionNumber'][0]
accessionNumber=r.json()['filings']['recent']['accessionNumber'][0].replace('-','')
accessionNumber_dash=r.json()['filings']['recent']['accessionNumber'][0]

print(accessionNumber)

https://data.sec.gov/submissions/CIK0001535392.json
dict_keys(['recent', 'files'])
dict_keys(['accessionNumber', 'filingDate', 'reportDate', 'acceptanceDateTime', 'act', 'form', 'fileNumber', 'filmNumber', 'items', 'size', 'isXBRL', 'isInlineXBRL', 'primaryDocument', 'primaryDocDescription'])
2022-11-14
2022-09-30
000117266122002482


In [202]:
cik_no_pad='1535392'
BASE_URL_FORM_TABLE = f"https://www.sec.gov/Archives/edgar/data/{cik_no_pad}/{accessionNumber}"
BASE_URL_FORM_TABLE = "https://www.sec.gov/Archives/edgar/data/1104186/000110418622000007/"
BASE_URL_FORM_TABLE= 'https://www.sec.gov/Archives/edgar/data/1535392/000117266122002482'
BASE_URL_FORM_TABLE = f'https://www.sec.gov/Archives/edgar/data/1535392/{accessionNumber_dash}.txt'

print(BASE_URL_FORM_TABLE)
headers = {'User-Agent':
           'StockFries stockfries@gmail.com'}
api_endpoint = BASE_URL_FORM_TABLE
print(api_endpoint)
r = requests.get(url=api_endpoint, headers=headers)
split_on = "/Archives/edgar/data/1535392/000117266122002482"
str(r.content)#.split(split_on)[-1]#.split('">')[0]

https://www.sec.gov/Archives/edgar/data/1535392/0001172661-22-002482.txt
https://www.sec.gov/Archives/edgar/data/1535392/0001172661-22-002482.txt


'b\'<SEC-DOCUMENT>0001172661-22-002482.txt : 20221114\\n<SEC-HEADER>0001172661-22-002482.hdr.sgml : 20221114\\n<ACCEPTANCE-DATETIME>20221114130630\\nACCESSION NUMBER:\\t\\t0001172661-22-002482\\nCONFORMED SUBMISSION TYPE:\\t13F-HR\\nPUBLIC DOCUMENT COUNT:\\t\\t2\\nCONFORMED PERIOD OF REPORT:\\t20220930\\nFILED AS OF DATE:\\t\\t20221114\\nDATE AS OF CHANGE:\\t\\t20221114\\nEFFECTIVENESS DATE:\\t\\t20221114\\n\\nFILER:\\n\\n\\tCOMPANY DATA:\\t\\n\\t\\tCOMPANY CONFORMED NAME:\\t\\t\\tMANGROVE PARTNERS\\n\\t\\tCENTRAL INDEX KEY:\\t\\t\\t0001535392\\n\\t\\tIRS NUMBER:\\t\\t\\t\\t980652572\\n\\t\\tSTATE OF INCORPORATION:\\t\\t\\tE9\\n\\t\\tFISCAL YEAR END:\\t\\t\\t1231\\n\\n\\tFILING VALUES:\\n\\t\\tFORM TYPE:\\t\\t13F-HR\\n\\t\\tSEC ACT:\\t\\t1934 Act\\n\\t\\tSEC FILE NUMBER:\\t028-15348\\n\\t\\tFILM NUMBER:\\t\\t221383114\\n\\n\\tBUSINESS ADDRESS:\\t\\n\\t\\tSTREET 1:\\t\\t645 MADISON AVENUE, 14TH FLOOR\\n\\t\\tCITY:\\t\\t\\tNEW YORK\\n\\t\\tSTATE:\\t\\t\\tNY\\n\\t\\tZIP:\\t\\t\\t10022\\n\

In [225]:
cik_no_pad='1535392'
BASE_URL_FORM_TABLE = f"https://www.sec.gov/Archives/edgar/data/{cik_no_pad}/{accessionNumber}/xslForm13F_X01/infotable.xml"
print(BASE_URL_FORM_TABLE)
BASE_URL_FORM_TABLE = f"https://www.sec.gov/Archives/edgar/data/{cik_no_pad}/{accessionNumber}/infotable.xml"
BASE_URL_FORM_TABLE = 'https://www.sec.gov/Archives/edgar/data/1628110/000090571821000700/infotable.xml'
BASE_URL_FORM_TABLE = f'https://www.sec.gov/Archives/edgar/data/1535392/{accessionNumber_dash}.txt'

#Test with excel.xml
#BASE_URL_FORM_TABLE = 'https://www.sec.gov/Archives/edgar/data/1491072/000149107222000007/excel.xml'

print(BASE_URL_FORM_TABLE)
headers = {'User-Agent':
           'StockFries stockfries@gmail.com'}
api_endpoint = BASE_URL_FORM_TABLE
print(api_endpoint)
r = requests.get(url=api_endpoint, headers=headers)
#r.json()
#xml_updated = ET.fromstring(str(r.content).replace('<shrsOrPrnAmt>','').replace('</shrsOrPrnAmt>',''))
#print(r.content)
#xml_updated = ET.fromstring(r.content)

#parser = ET.XMLParser(encoding="utf-8")
#tree = ET.parse(r.content, parser=parser)
#print(xml_updated)

#pandas.io.json.json_normalize

#xml_doc = xmltodict.parse(r.content)
#print(xml_doc)
#xml_doc['informationTable']['infoTable']
#xml_doc['ns1:informationTable']['ns1:infoTable']
response_text = r.text.split('<XML>\n')[2].split('\n</XML>')[0]#.replace('\n','')
xml_doc = xmltodict.parse(response_text)
xml_doc['informationTable']

https://www.sec.gov/Archives/edgar/data/1535392/000117266122002482/xslForm13F_X01/infotable.xml
https://www.sec.gov/Archives/edgar/data/1535392/0001172661-22-002482.txt
https://www.sec.gov/Archives/edgar/data/1535392/0001172661-22-002482.txt


OrderedDict([('@xsi:schemaLocation',
              'http://www.sec.gov/edgar/document/thirteenf/informationtable eis_13FDocument.xsd'),
             ('@xmlns',
              'http://www.sec.gov/edgar/document/thirteenf/informationtable'),
             ('@xmlns:n1',
              'http://www.sec.gov/edgar/document/thirteenf/informationtable'),
             ('@xmlns:xsi', 'http://www.w3.org/2001/XMLSchema-instance'),
             ('infoTable',
              [OrderedDict([('nameOfIssuer', '10X CAPITAL VENTURE ACQ III'),
                            ('titleOfClass', '*W EXP 06/30/202'),
                            ('cusip', 'G87077114'),
                            ('value', '3606'),
                            ('shrsOrPrnAmt',
                             OrderedDict([('sshPrnamt', '357045'),
                                          ('sshPrnamtType', 'SH')])),
                            ('investmentDiscretion', 'SOLE'),
                            ('votingAuthority',
                    

In [226]:
#https://stackoverflow.com/questions/40154727/how-to-use-xmltodict-to-get-items-out-of-an-xml-file


df = pd.DataFrame(xml_doc['informationTable']['infoTable'])
xml_doc['informationTable']['infoTable']
sshPrnamtType_list = []
sshPrnamt_list = []
putCall_list = []
for element in xml_doc['informationTable']['infoTable']:
    #print(element)
    sshPrnamt_list.append(element['shrsOrPrnAmt']['sshPrnamt'])
    putCall_list.append(element['shrsOrPrnAmt']['sshPrnamt'])
    #df_shares=pd.DataFrame(xml_doc['informationTable']['infoTable']['shrsOrPrnAmt'])
#df_shares.tail()
df['sshPrnamt']= sshPrnamt_list
#df['sshPrnamtType'] = sshPrnamtType_list
df.tail()

,nameOfIssuer,titleOfClass,cusip,value,shrsOrPrnAmt,investmentDiscretion,votingAuthority,sshPrnamt
368,XPO LOGISTICS INC,COM,983793100,14246,"{'sshPrnamt': '320002', 'sshPrnamtType': 'SH'}",SOLE,"{'Sole': '320002', 'Shared': '0', 'None': '0'}",320002
369,YOTTA ACQUISITION CORPORATIO,RIGHT 03/15/2027,98741Y129,117,"{'sshPrnamt': '1950000', 'sshPrnamtType': 'SH'}",SOLE,"{'Sole': '1950000', 'Shared': '0', 'None': '0'}",1950000
370,YOTTA ACQUISITION CORPORATIO,*W EXP 03/15/202,98741Y111,9396,"{'sshPrnamt': '950000', 'sshPrnamtType': 'SH'}",SOLE,"{'Sole': '950000', 'Shared': '0', 'None': '0'}",950000
371,ZIMMER ENERGY TRANSITION ACQ,COM CL A,989570106,149,"{'sshPrnamt': '15309', 'sshPrnamtType': 'SH'}",SOLE,"{'Sole': '15309', 'Shared': '0', 'None': '0'}",15309
372,Z-WORK ACQUISITION CORP,*W EXP 01/04/202,98880C110,322,"{'sshPrnamt': '32656', 'sshPrnamtType': 'SH'}",SOLE,"{'Sole': '32656', 'Shared': '0', 'None': '0'}",32656


In [254]:
def create_holdings_df(cik_='1535392',fund_name_='Mangrove Partners'):
    #Pull Filing accessionNumbers
    headers_ = {'User-Agent':
           'StockFries stockfries@gmail.com'}
    cik_padded_ = cik_.rjust(10, "0")#'0001535392'
    url_submissions = f'https://data.sec.gov/submissions/CIK{cik_padded_}.json'
    print(url_submissions)
    r_ = requests.get(url=url_submissions, headers=headers_)

    #Pull Latest 13F-HR Filing
    new_13F_level='None'
    old_13F_level='None'
    for index, filing_form in enumerate(r_.json()['filings']['recent']['form']):
        if (filing_form=='13F-HR') & (new_13F_level=='None'):
            new_13F_level = index
            continue
        elif ((filing_form=='13F-HR') & (new_13F_level!='None') & (old_13F_level=='None')):
            old_13F_level = index
            break
        #elif (new_13F_level!='None') & (new_13F_level!='None'):
        #    break
        else:
            continue
    #print('new_13F_level', 'old_13F_level')
    #print(new_13F_level, old_13F_level)
    
            
    new_accessionNumber=r_.json()['filings']['recent']['accessionNumber'][new_13F_level].replace('-','')
    new_accessionNumber_dash=r_.json()['filings']['recent']['accessionNumber'][new_13F_level]
    new_filingDate = r_.json()['filings']['recent']['filingDate'][new_13F_level]
    new_reportDate = r_.json()['filings']['recent']['reportDate'][new_13F_level]
    new_form = r_.json()['filings']['recent']['form'][new_13F_level]
    old_accessionNumber=r_.json()['filings']['recent']['accessionNumber'][old_13F_level].replace('-','')
    old_accessionNumber_dash=r_.json()['filings']['recent']['accessionNumber'][old_13F_level]
    old_filingDate = r_.json()['filings']['recent']['filingDate'][old_13F_level]
    old_reportDate = r_.json()['filings']['recent']['reportDate'][old_13F_level]
    old_form = r_.json()['filings']['recent']['form'][old_13F_level]
    
    BASE_URL_FORM_TABLE = f'https://www.sec.gov/Archives/edgar/data/{cik_}/{new_accessionNumber_dash}.txt'
    print(BASE_URL_FORM_TABLE)
    r_filing_new = requests.get(url=BASE_URL_FORM_TABLE, headers=headers_)
    response_text_new = r_filing_new.text.split('<XML>\n')[2].split('\n</XML>')[0]#.replace('\n','')
    xml_doc_ = xmltodict.parse(response_text_new)
    df_new = pd.DataFrame(xml_doc_['informationTable']['infoTable'])
    
    sshPrnamtType_lists = []
    sshPrnamt_lists = []
    for element in xml_doc_['informationTable']['infoTable']:
        #print(element['shrsOrPrnAmt']['sshPrnamt'])
        sshPrnamt_lists.append(int(element['shrsOrPrnAmt']['sshPrnamt']))
        sshPrnamtType_lists.append(element['shrsOrPrnAmt']['sshPrnamtType'])
    df_new['sshPrnamt']= sshPrnamt_lists
    df_new['sshPrnamtType'] = sshPrnamtType_lists
    df_new['cik']=cik_
    df_new['filingDate']= new_filingDate
    df_new['reportDate']= new_reportDate
    
    #Pull Previous Filing
    BASE_URL_FORM_TABLE = f'https://www.sec.gov/Archives/edgar/data/{cik_}/{old_accessionNumber_dash}.txt'
    print(BASE_URL_FORM_TABLE)
    r_filing_old = requests.get(url=BASE_URL_FORM_TABLE, headers=headers_)
    response_text_old = r_filing_old.text.split('<XML>\n')[2].split('\n</XML>')[0]#.replace('\n','')
    xml_doc_ = xmltodict.parse(response_text_old)
    df_old = pd.DataFrame(xml_doc_['informationTable']['infoTable'])
    
    sshPrnamtType_lists = []
    sshPrnamt_lists = []
    for element in xml_doc_['informationTable']['infoTable']:
        #print(element['shrsOrPrnAmt']['sshPrnamt'])
        sshPrnamt_lists.append(int(element['shrsOrPrnAmt']['sshPrnamt']))
        sshPrnamtType_lists.append(element['shrsOrPrnAmt']['sshPrnamtType'])
    df_old['sshPrnamt']= sshPrnamt_lists
    df_old['sshPrnamtType'] = sshPrnamtType_lists

    df_old['cik']=cik_
    df_old['filingDate']= old_filingDate
    df_old['reportDate']= old_reportDate
    df_final = pd.concat([df_new, df_old])
    df_final['fund_name'] = fund_name_
    df_final['value'] = df_final['value'].astype('int')
    return df_final

In [255]:
df = create_holdings_df()

df.tail()

https://data.sec.gov/submissions/CIK0001535392.json
https://www.sec.gov/Archives/edgar/data/1535392/0001172661-22-002482.txt
https://www.sec.gov/Archives/edgar/data/1535392/0001172661-22-001938.txt


,nameOfIssuer,titleOfClass,cusip,value,shrsOrPrnAmt,investmentDiscretion,votingAuthority,sshPrnamt,sshPrnamtType,cik,filingDate,reportDate,fund_name
515,WINVEST ACQUISITION CORP,*W EXP 08/09/202,97655B117,4843,"{'sshPrnamt': '974500', 'sshPrnamtType': 'SH'}",SOLE,"{'Sole': '974500', 'Shared': '0', 'None': '0'}",974500,SH,1535392,2022-08-15,2022-06-30,Mangrove Partners
516,XPAC ACQUISITION CORP,CLASS A ORD,G9831X106,4880,"{'sshPrnamt': '500000', 'sshPrnamtType': 'SH'}",SOLE,"{'Sole': '500000', 'Shared': '0', 'None': '0'}",500000,SH,1535392,2022-08-15,2022-06-30,Mangrove Partners
517,YOTTA ACQUISITION CORPORATIO,UNIT 03/15/2027,98741Y202,4487,"{'sshPrnamt': '450000', 'sshPrnamtType': 'SH'}",SOLE,"{'Sole': '450000', 'Shared': '0', 'None': '0'}",450000,SH,1535392,2022-08-15,2022-06-30,Mangrove Partners
518,YOTTA ACQUISITION CORPORATIO,RIGHT 03/15/2027,98741Y129,120,"{'sshPrnamt': '1500000', 'sshPrnamtType': 'SH'}",SOLE,"{'Sole': '1500000', 'Shared': '0', 'None': '0'}",1500000,SH,1535392,2022-08-15,2022-06-30,Mangrove Partners
519,ZENDESK INC,COM,98936J101,11701,"{'sshPrnamt': '157974', 'sshPrnamtType': 'SH'}",SOLE,"{'Sole': '157974', 'Shared': '0', 'None': '0'}",157974,SH,1535392,2022-08-15,2022-06-30,Mangrove Partners


## Process Data

In [256]:
#print(fund_dict.keys())
df_all = pd.DataFrame()
for fund in fund_dict:
    try:
        fund_name = fund
        cik = fund_dict[fund].split('CIK=')[1].split('&')[0]
        print(fund_name, cik)
        df_fund = create_holdings_df(cik_=cik, fund_name_=fund_name)
        df_all = pd.concat([df_all, df_fund]).reset_index(drop=True)
    except:
        print('Error')
        print(traceback.format_exc())
        
        continue
print(df_all.columns)
df_all.tail()

ABRAMS CAPITAL MANAGEMENT, L.P. 1358706
https://data.sec.gov/submissions/CIK0001358706.json
https://www.sec.gov/Archives/edgar/data/1358706/0001172661-22-002366.txt
https://www.sec.gov/Archives/edgar/data/1358706/0001172661-22-001829.txt
AKRE CAPITAL MANAGEMENT 1112520
https://data.sec.gov/submissions/CIK0001112520.json
https://www.sec.gov/Archives/edgar/data/1112520/0001112520-22-000015.txt
https://www.sec.gov/Archives/edgar/data/1112520/0001112520-22-000013.txt
ANSON FUNDS 1491072
https://data.sec.gov/submissions/CIK0001491072.json
https://www.sec.gov/Archives/edgar/data/1491072/0001491072-22-000007.txt
Error
Traceback (most recent call last):
  File "/var/folders/yb/_9tm4m2j26x6mhfjhf4r1v140000gn/T/ipykernel_12429/1473845845.py", line 8, in <module>
    df_fund = create_holdings_df(cik_=cik, fund_name_=fund_name)
  File "/var/folders/yb/_9tm4m2j26x6mhfjhf4r1v140000gn/T/ipykernel_12429/3586383357.py", line 44, in create_holdings_df
    df_new = pd.DataFrame(xml_doc_['informationTable

,nameOfIssuer,titleOfClass,cusip,value,shrsOrPrnAmt,investmentDiscretion,votingAuthority,sshPrnamt,sshPrnamtType,cik,filingDate,reportDate,fund_name,putCall,otherManager
9712,TWELVE SEAS INVESTMENT CO II,*W EXP 03/02/202,90118T114,16,"{'sshPrnamt': '190848', 'sshPrnamtType': 'SH'}",SOLE,"{'Sole': '190848', 'Shared': '0', 'None': '0'}",190848,SH,1348145,2022-08-15,2022-06-30,VR ADVISORY SERVICES,NaN,NaN
9713,VELO3D INC,*W EXP 12/01/202,92259N112,34,"{'sshPrnamt': '100000', 'sshPrnamtType': 'SH'}",SOLE,"{'Sole': '100000', 'Shared': '0', 'None': '0'}",100000,SH,1348145,2022-08-15,2022-06-30,VR ADVISORY SERVICES,NaN,NaN
9714,VIEW INC,*W EXP 03/08/202,92671V114,16,"{'sshPrnamt': '105000', 'sshPrnamtType': 'SH'}",SOLE,"{'Sole': '105000', 'Shared': '0', 'None': '0'}",105000,SH,1348145,2022-08-15,2022-06-30,VR ADVISORY SERVICES,NaN,NaN
9715,WARRIOR TECHNOLOGIES ACQUI C,*W EXP 03/31/202,936273119,8,"{'sshPrnamt': '100000', 'sshPrnamtType': 'SH'}",SOLE,"{'Sole': '100000', 'Shared': '0', 'None': '0'}",100000,SH,1348145,2022-08-15,2022-06-30,VR ADVISORY SERVICES,NaN,NaN
9716,YANDEX N V,SHS CLASS A,N97284108,1313,"{'sshPrnamt': '429023', 'sshPrnamtType': 'SH'}",SOLE,"{'Sole': '429023', 'Shared': '0', 'None': '0'}",429023,SH,1348145,2022-08-15,2022-06-30,VR ADVISORY SERVICES,NaN,NaN


In [251]:
print('investmentDiscretion', set(df.investmentDiscretion.values))
#print('shrsOrPrnAmt', set(df.shrsOrPrnAmt.values))
periods = list(set(df_all.reportDate.values))
print(periods)
df_all.reset_index().tail()
#df_all = pd.to_numeric(df_all)
df_all[df_all.reportDate == periods[0]].reset_index().tail()
df_current = df_all[df_all.reportDate == max(periods)].reset_index(drop=True)
df_previous = df_all[df_all.reportDate == min(periods)].reset_index(drop=True)
df_current.to_csv('df_current.csv')
df_previous.to_csv('df_previous.csv')
df_previous
df_previous.groupby(['nameOfIssuer','cusip']).sum()
#df_previous.groupby(['nameOfIssuer','cusip']).agg()
df_current.groupby(['nameOfIssuer','cusip']).sum()

investmentDiscretion {'SOLE'}
['2017-06-30', '2019-12-31', '2019-06-30', '2022-09-30', '2018-06-30', '2020-03-31', '2020-12-31', '2020-06-30', '2019-09-30', '2017-09-30', '2018-09-30', '2022-06-30', '2019-03-31', '2021-03-31', '2018-12-31', '2020-09-30']


/var/folders/yb/_9tm4m2j26x6mhfjhf4r1v140000gn/T/ipykernel_12429/3770244570.py:13: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_previous.groupby(['nameOfIssuer','cusip']).sum()
/var/folders/yb/_9tm4m2j26x6mhfjhf4r1v140000gn/T/ipykernel_12429/3770244570.py:15: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_current.groupby(['nameOfIssuer','cusip']).sum()


value  sshPrnamt
nameOfIssuer                 cusip                       
10X CAPITAL VENTURE ACQ III  G87077114  151452   14995890
8I ACQUISITION 2 CORP        G2956R129   31962   66623172
A SPAC I ACQUISITION CORP    G0542S114    7560   63000000
                             G0542S130  210210   21000000
A SPAC II ACQUISITION CORP   G0543H109  160734   16152360
...                                        ...        ...
XPO LOGISTICS INC            983793100  598332   13440084
YOTTA ACQUISITION CORPORATIO 98741Y111  394632   39900000
                             98741Y129    4914   81900000
Z-WORK ACQUISITION CORP      98880C110   13524    1371552
ZIMMER ENERGY TRANSITION ACQ 989570106    6258     642978

[373 rows x 2 columns]

In [98]:
655256*1000/8846544

74.06915061972224

In [250]:
df_current_g = df_current.groupby(['nameOfIssuer','cusip']).sum()#.sort_values('value', ascending=False)
df_previous_g = df_previous.groupby(['nameOfIssuer','cusip']).sum()#.sort_values('value', ascending=False)
df_current_g.to_csv('df_current_g.csv')
df_previous_g.to_csv('df_previous_g.csv')
df_previous_g.tail()
df_diff = df_current_g - df_previous_g
df_diff.sort_values('value').value.max()

/var/folders/yb/_9tm4m2j26x6mhfjhf4r1v140000gn/T/ipykernel_12429/1850444649.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_current_g = df_current.groupby(['nameOfIssuer','cusip']).sum()#.sort_values('value', ascending=False)
/var/folders/yb/_9tm4m2j26x6mhfjhf4r1v140000gn/T/ipykernel_12429/1850444649.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_previous_g = df_previous.groupby(['nameOfIssuer','cusip']).sum()#.sort_values('value', ascending=False)


1980956

In [252]:
def process_scraped_data(df_all_):
    periods = list(set(df_all_.reportDate.values))
    #print(periods[0])
    df_all_.reset_index().tail()
    df_all_[df_all.reportDate == periods[0]].reset_index().tail()
    df_current = df_all[df_all.reportDate == max(periods)].reset_index(drop=True)
    df_previous = df_all[df_all.reportDate == min(periods)].reset_index(drop=True)
    df_current_g = df_current.groupby(['nameOfIssuer','cusip']).sum().sort_values('value', ascending=False)
    df_previous_g = df_previous.groupby(['nameOfIssuer','cusip']).sum().sort_values('value', ascending=False)
    
    df_diff = df_current_g - df_previous_g
    df_heavy_ = df_current_g
    df_hot_ = df_diff[df_diff.value > 0]
    df_cold_ = df_diff[df_diff.value < 0]
    return df_heavy_, df_hot_, df_cold_
    

## Full Flow

In [253]:
df_all = pd.DataFrame()
for fund in fund_dict:
    try:
        fund_name = fund
        cik = fund_dict[fund].split('CIK=')[1].split('&')[0]
        df_fund = create_holdings_df()
        df_all = pd.concat([df_all, df_fund]).reset_index(drop=True)
        print(fund_name, cik)
    except:
        continue
df_heavy, df_hot, df_cold = process_scraped_data(df_all)

https://data.sec.gov/submissions/CIK0001535392.json
https://www.sec.gov/Archives/edgar/data/1535392/0001172661-22-002482.txt
https://www.sec.gov/Archives/edgar/data/1535392/0001172661-22-001938.txt
ABRAMS CAPITAL MANAGEMENT, L.P. 1358706
https://data.sec.gov/submissions/CIK0001535392.json
https://www.sec.gov/Archives/edgar/data/1535392/0001172661-22-002482.txt
https://www.sec.gov/Archives/edgar/data/1535392/0001172661-22-001938.txt
AKRE CAPITAL MANAGEMENT 1112520
https://data.sec.gov/submissions/CIK0001535392.json
https://www.sec.gov/Archives/edgar/data/1535392/0001172661-22-002482.txt
https://www.sec.gov/Archives/edgar/data/1535392/0001172661-22-001938.txt
ANSON FUNDS 1491072
https://data.sec.gov/submissions/CIK0001535392.json
https://www.sec.gov/Archives/edgar/data/1535392/0001172661-22-002482.txt
https://www.sec.gov/Archives/edgar/data/1535392/0001172661-22-001938.txt
APPALOOSA LP 1656456
https://data.sec.gov/submissions/CIK0001535392.json
https://www.sec.gov/Archives/edgar/data/153

/var/folders/yb/_9tm4m2j26x6mhfjhf4r1v140000gn/T/ipykernel_12429/3568157170.py:8: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_current_g = df_current.groupby(['nameOfIssuer','cusip']).sum().sort_values('value', ascending=False)
/var/folders/yb/_9tm4m2j26x6mhfjhf4r1v140000gn/T/ipykernel_12429/3568157170.py:9: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_previous_g = df_previous.groupby(['nameOfIssuer','cusip']).sum().sort_values('value', ascending=False)


In [62]:
import requests
from xml.etree import ElementTree
tree = ElementTree.fromstring(r.content)

In [9]:
#cik = validate_cik(cik)
api_endpoint = f"{BASE_URL_SUBMISSIONS}/CIK{cik}.json"
print('https://data.sec.gov/submissions/CIK##########.json')
submissions = self._rate_limited_get(api_endpoint)

filings = submissions["filings"]
paginated_submissions = filings["files"]

# Handle pagination for a large number of requests
if handle_pagination and paginated_submissions:
    to_merge = [filings["recent"]]
    for submission in paginated_submissions:
        filename = submission["name"]
        api_endpoint = f"{BASE_URL_SUBMISSIONS}/{filename}"
        resp = self._rate_limited_get(api_endpoint)
        to_merge.append(resp)

    # Merge all paginated submissions from files key into recent
    # and clear files list.
    filings["recent"] = merge_submission_dicts(to_merge)
    filings["files"] = []


https://data.sec.gov/submissions/CIK##########.json


NameError: name 'self' is not defined

## Parking Lot

In [ ]:
def create_holdings_df(cik_='1535392',fund_name_='Mangrove Partners'):
    #Pull Filing accessionNumbers
    headers_ = {'User-Agent':
           'StockFries stockfries@gmail.com'}
    cik_padded_ = cik_.rjust(10, "0")#'0001535392'
    url_submissions = f'https://data.sec.gov/submissions/CIK{cik_padded_}.json'
    print(url_submissions)
    r_ = requests.get(url=url_submissions, headers=headers)

    #Pull Latest 13F-HR Filing
    new_13F_level='None'
    old_13F_level='None'
    for index, filing_form in enumerate(r_.json()['filings']['recent']['form']):
        if (filing_form=='13F-HR') & (new_13F_level=='None'):
            new_13F_level = index
            continue
        elif ((filing_form=='13F-HR') & (new_13F_level!='None') & (old_13F_level=='None')):
            old_13F_level = index
            break
        #elif (new_13F_level!='None') & (new_13F_level!='None'):
        #    break
        else:
            continue
    #print('new_13F_level', 'old_13F_level')
    #print(new_13F_level, old_13F_level)
    
            
    new_accessionNumber=r_.json()['filings']['recent']['accessionNumber'][new_13F_level].replace('-','')
    new_filingDate = r_.json()['filings']['recent']['filingDate'][new_13F_level]
    new_reportDate = r_.json()['filings']['recent']['reportDate'][new_13F_level]
    new_form = r_.json()['filings']['recent']['form'][new_13F_level]
    old_accessionNumber=r_.json()['filings']['recent']['accessionNumber'][old_13F_level].replace('-','')
    old_filingDate = r_.json()['filings']['recent']['filingDate'][old_13F_level]
    old_reportDate = r_.json()['filings']['recent']['reportDate'][old_13F_level]
    old_form = r_.json()['filings']['recent']['form'][old_13F_level]
    
        
    #Pull Filing 13F-HR URL Extension
    url_filling_page_new = f"https://www.sec.gov/Archives/edgar/data/{cik_}/{new_accessionNumber}"
    print('url_filling_page_new: ', url_filling_page_new)
    r_filling_page_new = requests.get(url=api_endpoint, headers=headers)
    split_on = f"/Archives/edgar/data/{cik_}/{new_accessionNumber}/"
    filing_extension = str(r_filling_page_new.content).split(split_on)[-1].split('">')[0]

    url_filling_new = f"https://www.sec.gov/Archives/edgar/data/{cik_}/{new_accessionNumber}/{filing_extension}"#infotable.xml
    print('split_on: ', split_on)
    print(new_reportDate, url_filling_new)
    
    r_filing_new = requests.get(url=url_filling_new, headers=headers)
    #df_new = pd.read_xml(r_filing_new.content)
    xml_doc_ = xmltodict.parse(r_filing_new.content)
    df_new = pd.DataFrame(xml_doc_['informationTable']['infoTable'])
    #xml_doc_['informationTable']['infoTable']
    
    sshPrnamtType_lists = []
    sshPrnamt_lists = []
    for element in xml_doc_['informationTable']['infoTable']:
        #print(element['shrsOrPrnAmt']['sshPrnamt'])
        sshPrnamt_lists.append(int(element['shrsOrPrnAmt']['sshPrnamt']))
        sshPrnamtType_lists.append(element['shrsOrPrnAmt']['sshPrnamtType'])
    df_new['sshPrnamt']= sshPrnamt_lists
    df_new['sshPrnamtType'] = sshPrnamtType_lists
    df_new['cik']=cik_
    df_new['filingDate']= new_filingDate
    df_new['reportDate']= new_reportDate
    
    #Pull Previous Filing
    url_filling_old = f"https://www.sec.gov/Archives/edgar/data/{cik_}/{old_accessionNumber}/infotable.xml"
    print(old_reportDate, url_filling_old)
    r_filing_old = requests.get(url=url_filling_old, headers=headers)
    #df_old = pd.read_xml(r_filing_old.content)
    #df_old = xmltodict.parse(r_filing_new.content)
    xml_doc_ = xmltodict.parse(r_filing_old.content)
    df_old = pd.DataFrame(xml_doc_['informationTable']['infoTable'])
    #xml_doc_['informationTable']['infoTable']
    
    sshPrnamtType_lists = []
    sshPrnamt_lists = []
    for element in xml_doc_['informationTable']['infoTable']:
        #print(element['shrsOrPrnAmt']['sshPrnamt'])
        sshPrnamt_lists.append(int(element['shrsOrPrnAmt']['sshPrnamt']))
        sshPrnamtType_lists.append(element['shrsOrPrnAmt']['sshPrnamtType'])
    df_old['sshPrnamt']= sshPrnamt_lists
    df_old['sshPrnamtType'] = sshPrnamtType_lists

    df_old['cik']=cik_
    df_old['filingDate']= old_filingDate
    df_old['reportDate']= old_reportDate
    df_final = pd.concat([df_new, df_old])
    df_final['fund_name'] = fund_name_
    df_final['value'] = df_final['value'].astype('int')
    return df_final